In [ ]:
from ase import Atoms, Atom
from ase.io import read, write
from ase.data import chemical_symbols, covalent_radii, vdw_alvarez
from ase.parallel import paropen as open

from pathlib import Path
import os
import numpy as np
from pymatgen.core import Element
from tqdm.auto import tqdm
import pandas as pd


from alignn.ff.ff import AlignnAtomwiseCalculator,default_path


model_path = default_path()
calc = AlignnAtomwiseCalculator(path=model_path, device='cuda')

model_name = 'ALIGNN'


In [ ]:


for symbol in tqdm(chemical_symbols):
    
    s = set([symbol])
    
    if 'X' in s:
        continue
        
    try:
        atom = Atom(symbol)
        rmin = covalent_radii[atom.number] * 0.95
        rvdw = vdw_alvarez.vdw_radii[atom.number] if atom.number < len(vdw_alvarez.vdw_radii) else np.nan 
        rmax = 3.1 * rvdw if not np.isnan(rvdw) else 6
        rstep = 0.01 #if rmin < 1 else 0.4

        a = 2 * rmax

        npts = int((rmax - rmin)/rstep)

        rs = np.linspace(rmin, rmax, npts)
        e = np.zeros_like(rs)

        da = symbol + symbol

        out_dir = Path(str(da))

        os.makedirs(out_dir, exist_ok=True)

        skip = 0
        
        element = Element(symbol)
        
        try:
            m = element.valence[1]
            if element.valence == (0, 2):
                m = 0
        except:
            m = 0
            
            
        r = rs[0]
        
        positions = [
            [a/2-r/2, a/2, a/2],
            [a/2+r/2, a/2, a/2],
        ]
            
        traj_fpath = out_dir / f"{model_name}.extxyz"

        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
            skip = len(traj)
            atoms = traj[-1]
        else:
            # Create the unit cell with two atoms
            atoms = Atoms(
                da, 
                positions=positions,
                # magmoms=magmoms,
                cell=[a, a+0.001, a+0.002], 
                pbc=True
            )
        
        print(atoms)

        calc = calc

        atoms.calc = calc

        for i, r in enumerate(tqdm(rs)):

            if i < skip:
                continue

            positions = [
                [a/2-r/2, a/2, a/2],
                [a/2+r/2, a/2, a/2],
            ]
            
            # atoms.set_initial_magnetic_moments(magmoms)
                
            atoms.set_positions(positions)

            e[i] = atoms.get_potential_energy()
            
            atoms.calc.results.update({
                "forces": atoms.get_forces()
            })

            write(traj_fpath, atoms, append="a")
    except Exception as e:
        print(e)


In [ ]:


df = pd.DataFrame(columns=['name', 'method', 'R', 'E', 'F', 'S^2'])

for symbol in tqdm(chemical_symbols):
    
    da = symbol + symbol
    
    out_dir = Path(da)
    
    traj_fpath = out_dir / f"{model_name}.extxyz"


    if traj_fpath.exists():
        traj = read(traj_fpath, index=":")
    else:
        continue
    
    Rs, Es, Fs, S2s = [], [], [], []
    for atoms in traj:
        
        vec = atoms.positions[1] - atoms.positions[0]
        r = np.linalg.norm(vec)
        e = atoms.get_potential_energy()
        f = np.inner(vec/r, atoms.get_forces()[1])
        # s2 = np.mean(np.power(atoms.get_magnetic_moments(), 2))
        
        Rs.append(r)
        Es.append(e)
        Fs.append(f)
        # S2s.append(s2)
        
    data = {
        'name': da,
        'method': 'ALIGNN',
        'R': Rs,
        'E': Es,
        'F': Fs,
        'S^2': S2s
    }

    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

json_fpath = 'homonuclear-diatomics.json'

df.to_json(json_fpath, orient='records')    

In [ ]:
df